In [ ]:
# PRE-PROCESSING (NO EXTRINSIC YET)

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(1)

In [34]:
songs = pd.read_csv('ThesisHits.csv', index_col=0)
data = songs
print(data)

                                                title  \
0                                           Kill Bill   
1                                              Mahika   
2                                           Anti-Hero   
3                                        Here With Me   
4                                               Ditto   
5                                       Made You Look   
6                        Wet The Bed (feat. Ludacris)   
7                           Unholy (feat. Kim Petras)   
8                                 Under The Influence   
9             Babalik Sa'yo (from "2 Good 2 Be True")   
10  Until I Found You (with Em Beihold) - Em Beiho...   
11                                      Midnight Rain   
12                                             Umaasa   
13                                             Fallen   
14                                        Die For You   
15                                          As It Was   
16                             

In [35]:
print(data.isnull().sum())

title              0
artist/s           0
danceability       0
key                0
loudness           0
speechiness        0
liveness           0
valence            0
tempo              0
duration_ms        0
stream_category    0
dtype: int64


In [36]:
print(data.dtypes)

title               object
artist/s            object
danceability       float64
key                  int64
loudness           float64
speechiness        float64
liveness           float64
valence            float64
tempo              float64
duration_ms          int64
stream_category      int64
dtype: object


In [37]:
data = data.astype({'key':'float'})
data = data.astype({'duration_ms':'float'})
print(data.dtypes)

title               object
artist/s            object
danceability       float64
key                float64
loudness           float64
speechiness        float64
liveness           float64
valence            float64
tempo              float64
duration_ms        float64
stream_category      int64
dtype: object


In [38]:
# split data
data = data.drop(columns=['title', 'artist/s'])
X_data =  data.values[:,:-1]
y_data = data.values[:,-1]

print(X_data.shape)
print(y_data.shape)

(50, 8)
(50,)


In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=1)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

X_train:  (37, 8)
y_train:  (37,)
X_test:  (13, 8)
y_test:  (13,)


In [40]:
print(X_train)

[[ 6.00000e-01  6.00000e+00 -6.12400e+00  4.52000e-02  1.89000e-01
   5.19000e-01  1.58078e+02  1.85573e+05]
 [ 6.76000e-01  7.00000e+00 -5.45500e+00  7.11000e-02  1.27000e-01
   5.22000e-01  1.71883e+02  2.05428e+05]
 [ 8.48000e-01  3.00000e+00 -7.72000e+00  1.02000e-01  8.79000e-02
   5.72000e-01  1.20009e+02  1.81840e+05]
 [ 8.13000e-01  0.00000e+00 -6.44700e+00  3.72000e-02  1.83000e-01
   6.51000e-01  1.10053e+02  1.75889e+05]
 [ 8.74000e-01  1.10000e+01 -1.11680e+01  8.61000e-02  9.59000e-02
   7.62000e-01  1.22001e+02  2.28689e+05]
 [ 8.98000e-01  5.00000e+00 -7.97300e+00  6.20000e-02  8.76000e-02
   7.27000e-01  1.19869e+02  1.81507e+05]
 [ 6.52000e-01  6.00000e+00 -4.09800e+00  4.06000e-02  1.47000e-01
   6.15000e-01  9.80790e+01  2.01807e+05]
 [ 5.45000e-01  5.00000e+00 -1.10010e+01  3.00000e-02  9.44000e-02
   3.49000e-01  1.24022e+02  2.68124e+05]
 [ 3.48000e-01  6.00000e+00 -6.42500e+00  3.14000e-02  9.59000e-02
   3.01000e-01  1.37619e+02  2.70594e+05]
 [ 5.72000e-01  7.0

In [44]:
# normalizing (read that z-scaling is standardscaler)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) #why only transform? saw this sa intsy mp

In [52]:
def compute_RMSE(predictions, actual):
    return np.sqrt(np.square(np.subtract(actual, predictions)).mean())
def compute_mae(predictions, actual):
    mae = np.absolute(np.subtract(actual, predictions)).mean()
    return mae

In [ ]:
# RANDOM FOREST

In [45]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
 RandomForestClassifier?

In [47]:
rfc = RandomForestClassifier()
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [48]:
hyperparameters = [
        {
        'n_estimators' : [50, 100, 200, 400, 600, 800, 1000, 1200, 1600, 1800],
        'max_depth' : [10, 30, 50, 100, 150, 200, 250, 300, 350, None],
        'min_samples_split' : [2, 4, 6, 10, 15, 20],
        'max_leaf_nodes' : [100, 300, 500, 700, 900, 1100, 1300, None]
    }
]

In [50]:
from sklearn.model_selection import RandomizedSearchCV
rsrfc = RandomizedSearchCV(rfc, hyperparameters, random_state=42)
rsrfc.fit(X_train, y_train)

C:\Users\angel\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions=[{'max_depth': [10, 30, 50, 100, 150,
                                                       200, 250, 300, 350,
                                                       None],
                                         'max_leaf_nodes': [100, 300, 500, 700,
                                                            900, 1100, 1300,
                                                            None],
                                         'min_samples_split': [2, 4, 6, 10, 15,
                                                               20],
                                         'n_estimators': [50, 100, 200, 400,
                                                          600, 800, 1000, 1200,
                                                          1600, 1800]}],
                   random_state=42)

In [51]:
best_params_rfc = rsrfc.best_params_
best_params_rfc

{'n_estimators': 200,
 'min_samples_split': 20,
 'max_leaf_nodes': 1300,
 'max_depth': 300}

In [53]:
best_model_rfc = rsrfc.best_estimator_

predictions_train_rfc = best_model_rfc.predict(X_train)

train_rmse_rfc = compute_RMSE(predictions_train_rfc, y_train)
print('Training RMSE: {}'.format(train_rmse_rfc))

train_mae_rfc = compute_mae(predictions_train_rfc, y_train)
print('Training MAE: {}'.format(train_mae_rfc))

predictions_test_rfc = best_model_rfc.predict(X_test)

test_rmse_rfc = compute_RMSE(predictions_test_rfc, y_test)
print('Test RMSE: {}'.format(test_rmse_rfc))

test_mae_rfc = compute_mae(predictions_test_rfc, y_test)
print('Test MAE: {}'.format(test_mae_rfc))

Training RMSE: 2.43288284118223
Training MAE: 1.7027027027027026
Test RMSE: 3.088439984597651
Test MAE: 2.6153846153846154


In [ ]:
# SVM

In [54]:
from sklearn.svm import SVC

In [55]:
SVC?

In [56]:
svm = SVC()
svm.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [57]:
hyperparameters = [
        {
        'C' : [0.1, 1, 100, 1000],
        'kernel' : ['rbf', 'poly', 'sigmoid', 'linear'],
        'degree' : [1, 2, 3, 4, 5, 6]
    }
]

In [58]:
rssvm = RandomizedSearchCV(svm, hyperparameters, random_state=42)
rssvm.fit(X_train, y_train)

C:\Users\angel\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


RandomizedSearchCV(estimator=SVC(),
                   param_distributions=[{'C': [0.1, 1, 100, 1000],
                                         'degree': [1, 2, 3, 4, 5, 6],
                                         'kernel': ['rbf', 'poly', 'sigmoid',
                                                    'linear']}],
                   random_state=42)

In [59]:
best_params_svm = rssvm.best_params_
best_params_svm

{'kernel': 'poly', 'degree': 3, 'C': 1}

In [60]:
best_model_svm = rssvm.best_estimator_

predictions_train_svm = best_model_svm.predict(X_train)

train_rmse_svm = compute_RMSE(predictions_train_svm, y_train)
print('Training RMSE: {}'.format(train_rmse_svm))

train_mae_svm = compute_mae(predictions_train_svm, y_train)
print('Training MAE: {}'.format(train_mae_svm))

predictions_test_svm = best_model_svm.predict(X_test)

test_rmse_svm = compute_RMSE(predictions_test_svm, y_test)
print('Test RMSE: {}'.format(test_rmse_svm))

test_mae_svm = compute_mae(predictions_test_svm, y_test)
print('Test MAE: {}'.format(test_mae_svm))

Training RMSE: 1.9382354750537296
Training MAE: 1.054054054054054
Test RMSE: 3.05085107923876
Test MAE: 2.5384615384615383


In [ ]:
# BINARY TREES

In [61]:
from sklearn.tree import DecisionTreeClassifier

In [62]:
DecisionTreeClassifier?

In [63]:
dt = DecisionTreeClassifier()
dt.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [64]:
hyperparameters = [
    {
        'min_impurity_decrease': [0.001, 0.01, 0.05, 0.1, 0.3, 0.5],
        'max_depth': [5, 10, 20, 30],
        'min_samples_split': [2, 4, 6, 10, 15, 20],
        'max_leaf_nodes': [3, 5, 10, 20, 50, 100]
    }
]

In [65]:
rsdt = RandomizedSearchCV(dt, hyperparameters, cv=5, random_state=42, n_iter=50)
rsdt.fit(X_train, y_train)

C:\Users\angel\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=50,
                   param_distributions=[{'max_depth': [5, 10, 20, 30],
                                         'max_leaf_nodes': [3, 5, 10, 20, 50,
                                                            100],
                                         'min_impurity_decrease': [0.001, 0.01,
                                                                   0.05, 0.1,
                                                                   0.3, 0.5],
                                         'min_samples_split': [2, 4, 6, 10, 15,
                                                               20]}],
                   random_state=42)

In [66]:
best_params_dt = rsdt.best_params_
best_params_dt

{'min_samples_split': 2,
 'min_impurity_decrease': 0.1,
 'max_leaf_nodes': 100,
 'max_depth': 5}

In [67]:
best_model_dt = rsdt.best_estimator_

predictions_train_dt = best_model_dt.predict(X_train)

train_rmse_dt = compute_RMSE(predictions_train_dt, y_train)
print('Training RMSE: {}'.format(train_rmse_dt))

train_mae_dt = compute_mae(predictions_train_dt, y_train)
print('Training MAE: {}'.format(train_mae_dt))

predictions_test_dt = best_model_dt.predict(X_test)

test_rmse_dt = compute_RMSE(predictions_test_dt, y_test)
print('Test RMSE: {}'.format(test_rmse_dt))

test_mae_dt = compute_mae(predictions_test_dt, y_test)
print('Test MAE: {}'.format(test_mae_dt))

Training RMSE: 2.43288284118223
Training MAE: 1.7027027027027026
Test RMSE: 3.088439984597651
Test MAE: 2.6153846153846154
